In [1]:
!pip install git+https://github.com/huggingface/transformers.git@88d960937c81a32bfb63356a2e8ecf7999619681 gradio

  Cloning https://github.com/huggingface/transformers.git (to revision 88d960937c81a32bfb63356a2e8ecf7999619681) to /tmp/pip-req-build-y1ughfrm
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-y1ughfrm
  Running command git rev-parse -q --verify 'sha^88d960937c81a32bfb63356a2e8ecf7999619681'
  Running command git fetch -q https://github.com/huggingface/transformers.git 88d960937c81a32bfb63356a2e8ecf7999619681
  Running command git checkout -q 88d960937c81a32bfb63356a2e8ecf7999619681
  Resolved https://github.com/huggingface/transformers.git to commit 88d960937c81a32bfb63356a2e8ecf7999619681
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import gradio as gr
from transformers import pipeline
model_name = "BioMistral/BioMistral-7B"  # Replace with the actual model name if different <button class="citation-flag" data-index="1">
qa_pipeline = pipeline("question-answering", model=model_name)

# Step 2: Define global variables for session history
session_history = []

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def diagnose_symptoms(symptoms):
    """
    Use the BioMistral model to generate potential diagnoses based on the provided symptoms.
    """
    if not symptoms.strip():
        return "Please provide a detailed description of your symptoms."

    try:
        prompt = f"Symptoms: {symptoms}\nPossible Diagnosis:"
        result = qa_pipeline(prompt, max_length=100, num_return_sequences=1)
        diagnosis = result[0]['generated_text'].replace(prompt, "").strip()
        return diagnosis
    except Exception as e:
        return f"An error occurred: {str(e)}"

In [ ]:
with gr.Blocks(theme=gr.themes.Soft()) as demo:  # Use a modern theme for better aesthetics <button class="citation-flag" data-index="7">
    gr.Markdown("# 🏥 Medical Diagnosis Tool")
    gr.Markdown("Enter your symptoms, and the app will provide potential diagnoses based on the input.")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### Input Section")
            symptoms_input = gr.Textbox(label="Describe Your Symptoms", placeholder="Type your symptoms here...", lines=4)
            submit_button = gr.Button("Get Diagnosis", variant="primary")  # Styled button <button class="citation-flag" data-index="7">

        with gr.Column(scale=2):
            gr.Markdown("### Output Section")
            diagnosis_output = gr.Textbox(label="Potential Diagnosis", interactive=False, lines=6, show_copy_button=True)  # Enhanced output box

    # Add a feedback mechanism
    with gr.Accordion("Feedback", open=False):  # Collapsible section for feedback <button class="citation-flag" data-index="7">
        feedback = gr.Textbox(label="Was the diagnosis helpful? Let us know!", placeholder="Your feedback helps us improve!")
        feedback_submit = gr.Button("Submit Feedback")
        feedback_output = gr.Textbox(label="Thank You!", value="We appreciate your feedback!", interactive=False, visible=False)

        def handle_feedback(feedback_text):
            if feedback_text.strip():
                return gr.update(visible=True)  # Show thank-you message
            return gr.update(visible=False)

        feedback_submit.click(handle_feedback, inputs=feedback, outputs=feedback_output)

    # Define the button click action
    submit_button.click(diagnose_symptoms, inputs=symptoms_input, outputs=diagnosis_output)


In [ ]:
if __name__ == "__main__":
    demo.launch()